<a href="https://colab.research.google.com/github/eliasd/speedpred/blob/master/CommaSpeedPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/drive')

%cd drive/My\ Drive/Creative/speedpred

! git pull

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/Creative/speedpred
Already up to date.


In [19]:
! git fetch origin master
! git reset --hard FETCH_HEAD
! git clean -df

From https://github.com/eliasd/speedpred
 * branch            master     -> FETCH_HEAD
HEAD is now at 9481eec Ensure frame shape
Removing __pycache__/


In [2]:
import tensorflow as tf

from dataset import get_pre_processed_dataset
from model import get_speed_prediction_network

### Getting the dataset

In [3]:
# Dataset size:
# if window_size = 4, then 5100 elements.
ds = get_pre_processed_dataset()

# Most of the time in this cell is spent counting 
# the number of elements in the dataset.
DS_SIZE = sum(1 for _ in ds)
print(f'Number of elements: {DS_SIZE}')

train_size = int(0.7 * DS_SIZE)
val_size = int(0.15 * DS_SIZE)
test_size = int(0.15 * DS_SIZE)
print(f'Train size: {train_size}')
print(f'Validation size: {val_size}')
print(f'Test size: {test_size}')

# Shuffle dataset.
ds = ds.shuffle(buffer_size=DS_SIZE)

# Get training set.
train_ds = ds.take(train_size)

# Get validation and test sets.
full_test_ds = ds.skip(train_size)
val_ds = full_test_ds.take(val_size)
test_ds = full_test_ds.skip(test_size)

Number of elements: 5100
Train size: 3570
Validation size: 765
Test size: 765


### Fitting the Model

In [4]:
model = get_speed_prediction_network()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss='mse',
              metrics=['mse'])

model.fit(train_ds.batch(2), epochs=10, validation_data=val_ds.batch(2))

Epoch 1/10
1785/1785 [==============================] - 1391s 779ms/step - loss: 71.8529 - mse: 71.8529 - val_loss: 66.8233 - val_mse: 66.8233
Epoch 2/10
1785/1785 [==============================] - 1371s 768ms/step - loss: 70.0655 - mse: 70.0655 - val_loss: 67.8599 - val_mse: 67.8599
Epoch 3/10
1785/1785 [==============================] - 1386s 777ms/step - loss: 69.0457 - mse: 69.0457 - val_loss: 71.4496 - val_mse: 71.4496
Epoch 4/10
1785/1785 [==============================] - 1360s 762ms/step - loss: 69.8068 - mse: 69.8068 - val_loss: 72.0687 - val_mse: 72.0687
Epoch 5/10
1785/1785 [==============================] - 1370s 768ms/step - loss: 69.2681 - mse: 69.2681 - val_loss: 67.0811 - val_mse: 67.0811
Epoch 6/10
1785/1785 [==============================] - 1358s 761ms/step - loss: 68.3799 - mse: 68.3799 - val_loss: 67.1379 - val_mse: 67.1379
Epoch 7/10
1785/1785 [==============================] - 1357s 760ms/step - loss: 68.7761 - mse: 68.7761 - val_loss: 67.1615 - val_mse: 67.1615

In [8]:
count = 0
for window, labels in test_ds:
  pred = model(tf.expand_dims(window, 0))
  print(pred)
  print(labels)
  
  count = count + 1
  if count >= 3:
    break

tf.Tensor([[6.989753  7.0605826 7.101199  6.9767113]], shape=(1, 4), dtype=float32)
tf.Tensor([6.254949 6.193792 6.271255 6.240515], shape=(4,), dtype=float64)
tf.Tensor([[8.9433775 8.966254  8.991715  8.946243 ]], shape=(1, 4), dtype=float32)
tf.Tensor([9.493545 9.49822  9.488301 9.489941], shape=(4,), dtype=float64)
tf.Tensor([[5.335653  5.362772  5.3680983 5.3161635]], shape=(1, 4), dtype=float32)
tf.Tensor([6.14101  5.961272 6.00578  6.004716], shape=(4,), dtype=float64)


In [9]:
count = 0
for window, labels in train_ds:
  pred = model(tf.expand_dims(window, 0))
  print(pred)
  print(labels)
  print()
  
  count = count + 1
  if count >= 3:
    break

tf.Tensor([[8.971958  9.067791  9.0306015 8.941922 ]], shape=(1, 4), dtype=float32)
tf.Tensor([10.515698 10.484941 10.502711 10.492929], shape=(4,), dtype=float64)

tf.Tensor([[23.42581  23.423414 23.445442 23.425186]], shape=(1, 4), dtype=float32)
tf.Tensor([22.102941 22.137825 22.14674  22.163254], shape=(4,), dtype=float64)

tf.Tensor([[11.690115  11.6531105 11.862762  11.551782 ]], shape=(1, 4), dtype=float32)
tf.Tensor([11.609733 11.639742 11.782996 11.812252], shape=(4,), dtype=float64)



In [10]:
! git status

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   dataset.py
	modified:   test_dataset.py

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	__pycache__/
	train.mp4
	train.txt

no changes added to commit (use "git add" and/or "git commit -a")
